In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression


## Data Pre-Processing 
- Read the feather file into a dataframe 
- Drop some columns that do we do not want as the independent variables (Xs)

In [107]:
# df = pd.read_csv('lending_club_clean_sample.csv')
# df = pd.read_feather('lending_club_smaller_sample.feather')

# Read the feather file 
df = pd.read_feather('lending_club_clean.feather')

See which values we can classify as a default or cannot repay the loan 

In [108]:
# Check what kind of status does our Y variable have 
print(df[['loan_status']].value_counts())

loan_status                                        
Fully Paid                                             1497783
Current                                                1031016
Charged Off                                             362548
Late (31-120 days)                                       16154
In Grace Period                                          10028
Late (16-30 days)                                         2719
Issued                                                    2062
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                    433
dtype: int64


In [109]:
# Convert description into a default or not default prediction 
default_description = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period",
            "Late (16-30 days)", "Late (31-120 days)"]
df['default'] = df['loan_status'].apply(lambda status: 1 if status in default_description else 0).astype(int) 

In [110]:
# Create a different dataframe for the Y variable 
y = df['default']

# Drop variables that indirectly predict the default variable
df = df.drop(['out_prncp','out_prncp_inv', 'total_pymnt', 
              'total_pymnt_inv','total_rec_int','total_rec_prncp','total_rec_late_fee',
              'recoveries','collection_recovery_fee','last_pymnt_amnt'], axis=1)



In [111]:
# Drop unnecessary variables 
df = df.drop(['id','default', 'revol_util' ,'sec_app_earliest_cr_line','loan_status', 'grade', 'sub_grade', 
              'emp_title', 'issue_d','url','title',
              'zip_code','earliest_cr_line','last_pymnt_d',
              'next_pymnt_d','last_credit_pull_d','hardship_type',
              'hardship_start_date','hardship_end_date',
              'payment_plan_start_date','hardship_loan_status','debt_settlement_flag'], axis=1)

# Drop the hardship plan variables, since these usually apply after the loan is issued 
df = df.drop(['hardship_status','hardship_flag', 'hardship_reason','hardship_amount','hardship_length',
              'hardship_dpd','hardship_payoff_balance_amount','hardship_last_payment_amount'], axis=1)

# Drop fico numbers that are pulled after the loan is issued 
df = df.drop(['last_fico_range_high','last_fico_range_low'], axis=1)


In [112]:
# Properly turn variables that are categorical into the correct data type
df = pd.concat([df,pd.get_dummies(df["emp_length"],drop_first=True, prefix='emp_length')],axis=1)
df = pd.concat([df,pd.get_dummies(df["addr_state"],drop_first=True, prefix='addr_state')],axis=1)
df = pd.concat([df,pd.get_dummies(df["home_ownership"],drop_first=True, prefix='home_ownership')],axis=1)
df = pd.concat([df,pd.get_dummies(df["verification_status"],drop_first=True, prefix='verification_status')],axis=1)
df = pd.concat([df,pd.get_dummies(df["pymnt_plan"],drop_first=True, prefix='pymnt_plan')],axis=1)
df = pd.concat([df,pd.get_dummies(df["purpose"],drop_first=True, prefix='purpose')],axis=1)
df = pd.concat([df,pd.get_dummies(df["initial_list_status"],drop_first=True, prefix='initial_list_status')],axis=1)
df = pd.concat([df,pd.get_dummies(df["application_type"],drop_first=True, prefix='application_type')],axis=1)
df = pd.concat([df,pd.get_dummies(df["verification_status_joint"],drop_first=True, prefix='verification_status_joint')],axis=1)

# drop the categorical columns as well 
df = df.drop(['emp_length', 'addr_state', 'home_ownership', 
              'verification_status', 'pymnt_plan','purpose','initial_list_status',
              'application_type','verification_status_joint'], axis=1)

column_names = df.columns
 

In [114]:
column_names

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low',
       ...
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'initial_list_status_w', 'application_type_Joint App',
       'verification_status_joint_Source Verified',
       'verification_status_joint_Verified'],
      dtype='object', length=175)

# Split the Test - Train Dataset 

In [115]:
# Split train test dataset 
X_train, X_OOS_test, y_train, y_OOS_test = train_test_split(df, y, test_size=0.20, random_state=66)
print(column_names)

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low',
       ...
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'initial_list_status_w', 'application_type_Joint App',
       'verification_status_joint_Source Verified',
       'verification_status_joint_Verified'],
      dtype='object', length=175)


## Employ LASSO to reduce initial dimension 
We want to use LASSO right now to reduce the dimension from almost 100 to only those that play a significant role in predicting our dependent variable. 

In [70]:
# We do the LASSO to determine which variables are important and which are not 
# lasso = linear_model.Lasso(alpha=1)
# lasso.fit(X_train, y_train)

In [71]:
# # Print Lasso Coefficients that is not 0 
# coefficients = dict(zip(X_train.columns, lasso.coef_))
# print("Lasso Coefficients:")
# lasso_picked_var = []
# for feature, coef in coefficients.items():
#     if coef != 0:
#         lasso_picked_var.append(feature)
#         print(f"{feature}: {coef}")

In [34]:
# # Variable Visualization

# # Sorting the DataFrame by the absolute value of the coefficients
# sorted_coefficients_df = coefficients_df[coefficients_df['Coefficient'] != 0].sort_values('abs_coef', ascending=False)

# # Drop the 'abs_coef' column as it's no longer needed for plotting
# sorted_coefficients_df.drop('abs_coef', axis=1, inplace=True)

# # Plotting all non-zero features, sorted by the absolute value of their coefficients
# plt.figure(figsize=(15, 10))
# plt.bar(sorted_coefficients_df['Feature'], sorted_coefficients_df['Coefficient'])
# plt.xlabel('Features')
# plt.ylabel('Coefficient Value')
# plt.title('Lasso Coefficients - Sorted by Absolute Value')

# # Rotate the feature names on x-axis to avoid overlap
# plt.xticks(rotation=90)

# # Use tight layout to ensure the labels fit into the figure area
# plt.tight_layout()

# # Show the plot
# plt.show()


## Picked only variables with coefficient > 0 from LASSO and Normalised the values 

In [116]:
# pick the X variables that LASSO tells us are significant
# X_train = X_train[lasso_picked_var]
# X_OOS_test = X_OOS_test[lasso_picked_var]


# Normalised the values 
scaler = StandardScaler()
# Fit and apply normalization on the training set
X_train = scaler.fit_transform(X_train)
# Apply the same normalization rules on the test set
X_OOS_test = scaler.transform(X_OOS_test)


#### Uncomment the codes below incase the python needs to restart to save time doing the LASSO again 

In [64]:

# Output these files onto a csv file just incase we need to load them in again :) 
# X_train.to_csv('X_train.csv', index=False) 
# X_OOS_test.to_csv('X_OOS_test.csv', index=False) 
# y_train.to_csv('y_train.csv', index=False) 
# y_OOS_test.to_csv('y_OOS_test.csv', index=False) 

# X_train = pd.read_csv('X_train.csv')
# y_train = pd.read_csv('y_train.csv')['default']
# X_OOS_test = pd.read_csv('X_OOS_test.csv')
# y_OOS_test = pd.read_csv('y_OOS_test.csv')['default']


## Undersampling 
Since we see that the data is highly imbalanced, we try to oversample the data set with random under sampler. 

In [7]:
# from imblearn.over_sampling import SMOTE

# solve error by downgrading to install scikit-learn==1.2.2 
# ''' steps : 
# 1. install pip 
# 2. uninstall sci-kit 
# 3. uninstall imblearn
# 4. install sci-kit 1.2.2 
# 5. install imblearn 
# '''

# sm = SMOTE(random_state=42, sampling_strategy=0.2)

In [117]:
# Undersampling 
from imblearn.under_sampling import RandomUnderSampler
# Undersample the dataset 
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

See if the data is now balance or not 

In [118]:
# Count the different values 
print(y_resampled.value_counts())

# -- Code to Inspect the data set -- 
# df_oversampled = X_resampled
# df_oversampled['Outcome_Variable'] = y_resampled
# df_oversampled
# fig, ax = plt.subplots(figsize=(10, 8))
# df_oversampled['Outcome_Variable'].value_counts().plot(kind='bar', ax=ax, fontsize=14)
# ax.set_title('Oversampled Dataset', fontsize=16)
# ax.set_ylabel('Observation counts', fontsize=14)
# ax.set_xlabel('Class', fontsize=14)
# plt.show()


0    313925
1    313925
Name: default, dtype: int64


## K-Fold Cross Validation Setup
We setup K-Fold cross validation for parameters fine-tuning in the models below as a 5 Fold. 

In [38]:
# Set up K-Fold Cross Validation 
n_splits = 5
shuffle = True
random_state = 809
cv = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
# plot = plot_cv_indices(cv, X_resampled, y_resampled, n_splits)

## Model 1 : Logistic Regression 

In [9]:
# Function to plot the K-Fold stuff, don't need to run it. 
def plot_cv_indices(cv, X, y, n_splits, lw=10):
    '''
    This function plots the Cross validation indices.
    '''
    
    fig, ax = plt.subplots(figsize = (15,8))
    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=plt.cm.coolwarm,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes
    ax.scatter(range(len(X)), [ii + 1.5] * len(X), c=y, marker='_', lw=lw, cmap=plt.cm.Paired)

    # Formatting
    yticklabels = list(range(n_splits)) + ['Class']
    ax.set(yticks=np.arange(n_splits+1) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [72]:
# Model 1 : Logistic Model - K-Flod Cross Validation 
logistic_model = LogisticRegression(solver='lbfgs', max_iter=300)
for train_index, test_index in cv.split(X_resampled):
    X_cv_train, X_cv_test, y_cv_train, y_cv_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index], y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    logistic_model.fit(X_cv_train, y_cv_train)
    #Cross-Validation Prediction Error
    score = logistic_model.score(X_cv_test, y_cv_test)
    print(score)

In [73]:
# Just the logit regression, and OOS Prediction 
logit = LogisticRegression(solver='lbfgs', max_iter=1000)
logit.fit(X_resampled, y_resampled)    
score_OOS = logit.score(X_OOS_test, y_OOS_test)
print ("Logistic Model score :" , score_OOS)
y_pred = logit.predict(X_OOS_test)
print("Logistic Model Recall : " , recall_score(y_OOS_test, y_pred))
print("Logistic Model Precision : ", precision_score(y_OOS_test,y_pred))
print("Probability if you only predict 0s ", 1-round(95/56867,3))

Logistic Model score : 0.8933941093729437
Logistic Model Recall :  0.8865697807362992
Logistic Model Precision :  0.5663082728121069
Probability if you only predict 0s  0.998




## Model 2 : Lasso 

In [10]:
# Model 2 : LASSO 
# Cross Validate the penalty term in lasso
cross_validate_result = {}
cross_validate_recall = {} 
cross_validate_precision = {} 

# Set Hyperparameter (Lambda) values to cross validate here !!!!! 
lambdas = [0.01, 0.1]
for lambda_value in lambdas:
    print('Training Lasso Lambda Value : ', lambda_value)
    accuracies = []
    recall_scores = [] 
    precision_scores = [] 
    lasso = linear_model.Lasso(alpha=lambda_value)
    for train_index, test_index in cv.split(X_resampled):
        # change to loc to define the rows in the dataframe 
        X_cv_train, X_cv_test, y_cv_train, y_cv_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index], y_resampled.iloc[train_index], y_resampled.iloc[test_index]
        lasso.fit(X_cv_train, y_cv_train)
        y_pred = lasso.predict(X_cv_test)
        y_pred_classification = [1 if x >= 0.5 else 0 for x in y_pred]

        #Cross-Validation Prediction Error
        score = lasso.score(X_cv_test, y_cv_test)
        accuracies.append(score)
        recall_scores.append(recall_score(y_cv_test, y_pred_classification))
        precision_scores.append(precision_score(y_cv_test,y_pred_classification))

    cross_validate_result[lambda_value] = (sum(accuracies)/len(accuracies))
    cross_validate_recall[lambda_value] = (sum(recall_scores)/len(recall_scores))
    cross_validate_precision[lambda_value] = (sum(precision_scores)/len(precision_scores))

    print("Lambda Value Accuracy : " + str(lambda_value) + " " + str((sum(accuracies)/len(accuracies))))
    print("Lambda Value Precision : " + str(lambda_value) + " " + str((sum(recall_scores)/len(recall_scores))))
    print("Lambda Value Recall : " + str(lambda_value) + " " + str((sum(precision_scores)/len(precision_scores))))

# At the end you'll see what the value of each LASSO paramter is
print('Accuracy : ', cross_validate_result)
print('Precision : ',cross_validate_precision)
print('Recall : ',cross_validate_recall)
print(max(cross_validate_result, key=cross_validate_result.get))
    

## RUN LASSO GRID SEARCH HERE

In [119]:
lasso = linear_model.Lasso()
param_grid = [{'alpha': [0.01, 0.1]}]
grid_cv = GridSearchCV(estimator=lasso, cv=cv, param_grid=param_grid, n_jobs = -1)
grid_cv.fit(X_resampled, y_resampled)


In [ ]:
print(grid_cv.best_estimator_)

### OLS vs Lasso vs Ridge regression

In [121]:
# OLS 
ols = linear_model.LinearRegression()
ols.fit(X_resampled, y_resampled)  
y_pred = ols.predict(X_OOS_test)  
# turn the continous value into classification via simple >= 0.5 is 1 
y_pred_classification = [1 if x >= 0.5 else 0 for x in y_pred]
print("OLS Model Recall : " , recall_score(y_OOS_test, y_pred_classification))
print("OLS Model Precision : ", precision_score(y_OOS_test,y_pred_classification))
print("--")

# LASSO 
lasso = linear_model.Lasso()
lasso.fit(X_resampled, y_resampled)
y_pred = lasso.predict(X_OOS_test)
# turn the continous value into classification via simple >= 0.5 is 1 
y_pred_classification = [1 if x >= 0.5 else 0 for x in y_pred]
print("LASSO score : " , )
print("LASSO Model Recall : " , recall_score(y_OOS_test, y_pred_classification))
print("LASSO Model Precision : ", precision_score(y_OOS_test,y_pred_classification))
print("--")

# RIDGE
ridge = linear_model.Ridge(alpha=1)
ridge.fit(X_resampled, y_resampled)
y_pred = ridge.predict(X_OOS_test)
# turn the continous value into classification via simple >= 0.5 is 1 
y_pred_classification = [1 if x >= 0.5 else 0 for x in y_pred]
print("Ridge Model Recall : " , recall_score(y_OOS_test, y_pred_classification))
print("Ridge Model Precision : ", precision_score(y_OOS_test,y_pred_classification))

OLS Model Recall :  0.6537259584847176
OLS Model Precision :  0.2295999179044211
--
LASSO score : 
LASSO Model Recall :  1.0
LASSO Model Precision :  0.13453791580570126
--
Ridge Model Recall :  0.6537640692090755
Ridge Model Precision :  0.22960510759044692


In [95]:
# print(lasso.coef_)
for i in range(len(lasso.coef_)):
    if lasso.coef_[i] != 0:
        print(column_names[i], ' : ' , lasso.coef_[i])


term  :  0.018662426388216857
int_rate  :  0.00870873338631026
installment  :  0.015738092473865796
fico_range_low  :  0.02614201489935365
fico_range_high  :  6.55564841904912e-05
total_acc  :  0.006238836166335537
last_fico_range_high  :  -0.3184816293436844
last_fico_range_low  :  0.011232610405121173
all_util  :  -0.013546735790559738
mo_sin_old_rev_tl_op  :  0.003949785481439945
num_tl_30dpd  :  -0.0008980208735122508
hardship_amount  :  0.016658633309988718
hardship_dpd  :  0.002796340164766021
verification_status_Verified  :  0.0040488859038436305
initial_list_status_w  :  -0.0004368563731105071
hardship_flag_Y  :  -0.03781383622130164
hardship_status_BROKEN  :  0.0004026019993319039
hardship_status_COMPLETE  :  0.007515947162645549



## Random Forest 

In [54]:

random_forest = RandomForestClassifier(n_estimators = 50, max_depth=2, random_state=0)
random_forest.fit(X_resampled, y_resampled)


RandomForestClassifier(max_depth=2, n_estimators=50, random_state=0)

In [55]:
y_pred = random_forest.predict(X_OOS_test)
print("RND Forest Model Recall : " , recall_score(y_OOS_test, y_pred))
print("RND Forest Precision : ", precision_score(y_OOS_test,y_pred))

RND Forest Model Recall :  0.8762036637109683
RND Forest Precision :  0.5303004674621725


## Random forest with LASSO selected variables (Haven't fix this part - code doesn't work)

In [62]:

# Pick the coefficients that is not zero from LASSO 
# coefficients = dict(zip(lasso.coef_))
# print("Lasso Coefficients:")
# lasso_picked_var = []
# for coef in coefficients.items():
#     if coef != 0:
#         lasso_picked_var.append(feature)
#         print(f"{feature}: {coef}")

In [43]:
# # Random forest + LASSO 
# X_train_rf_lasso = X_resampled[lasso_picked_var]
# rf_lasso = RandomForestClassifier(max_depth=2)
# rf_lasso.fit(X_train_rf_lasso, y_resampled)

RandomForestClassifier(max_depth=2)

In [63]:
# X_test_rf_lasso_OOS = X_OOS_test[lasso_picked_var]
# y_pred = rf_lasso.predict(X_test_rf_lasso_OOS)
# print("RND Forest with LASSO Model Recall : " , recall_score(y_OOS_test, y_pred))
# print("RND Forest with LASSO Precision : ", precision_score(y_OOS_test,y_pred))

## Using Param Grid to search for best value with K-Fold Cross Validation 

In [128]:

# param_grid = [{'n_estimators': [200,225,250], 'max_depth' :[10,20,30]}]
# random_forest_cv = RandomForestClassifier()
# grid_cv = GridSearchCV(estimator=random_forest_cv, cv=cv, param_grid=param_grid, n_jobs = 3)
# grid_cv.fit(X_train_rf_lasso, y_resampled)

GridSearchCV(cv=KFold(n_splits=5, random_state=809, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=3,
             param_grid=[{'max_depth': [10, 20, 30],
                          'n_estimators': [200, 225, 250]}])

In [129]:
# print(grid_cv.best_estimator_)


RandomForestClassifier(max_depth=30, n_estimators=250)
